In [1]:
from pulp import *

# Define problem and variables
prob = LpProblem("NurseSchedulingProblem", LpMinimize)# 問題を定義し、最小化を目指すことを設定
days = ['day 1', 'day 2', 'day 3', 'day 4', 'day 5', 'day 6', 'day 7', 'day 8', 'day 9', 'day 10']# 日のリスト
nurses = ['Nurse 1', 'Nurse 2', 'Nurse 3', 'Nurse 4', 'Nurse 5', 'Nurse 6', 'Nurse 7', 'Nurse 8', 'Nurse 9', 'Nurse 10']# ナースのリスト
shifts = ['Day', 'Night', 'Late Night'] # シフトのリスト
shift_requirements = {'Day': 3, 'Night': 2, 'Late Night': 1}# 各シフト毎の勤務人数
workday_requirements = {'Nurse 1': 5, 'Nurse 2': 5, 'Nurse 3': 5, 'Nurse 4': 5, 'Nurse 5': 5, 'Nurse 6': 5, 'Nurse 7': 5, 'Nurse 8': 5, 'Nurse 9': 5, 'Nurse 10': 5}# 各ナースの必要な勤務日数


In [2]:
# Create decision variables
shifts_worked = LpVariable.dicts("ShiftsWorked", (nurses, days, shifts), cat='Binary')# 各ナースが各日の各シフトに割り当てられるかどうか

# Define objective function
prob += lpSum([shifts_worked[nurse][day][shift] for nurse in nurses for day in days for shift in shifts])# 目的関数を定義し、各ナースが働くシフトの総和を最小化

print(prob)# 変数と目的関数を表示

NurseSchedulingProblem:
MINIMIZE
1*ShiftsWorked_Nurse_10_day_10_Day + 1*ShiftsWorked_Nurse_10_day_10_Late_Night + 1*ShiftsWorked_Nurse_10_day_10_Night + 1*ShiftsWorked_Nurse_10_day_1_Day + 1*ShiftsWorked_Nurse_10_day_1_Late_Night + 1*ShiftsWorked_Nurse_10_day_1_Night + 1*ShiftsWorked_Nurse_10_day_2_Day + 1*ShiftsWorked_Nurse_10_day_2_Late_Night + 1*ShiftsWorked_Nurse_10_day_2_Night + 1*ShiftsWorked_Nurse_10_day_3_Day + 1*ShiftsWorked_Nurse_10_day_3_Late_Night + 1*ShiftsWorked_Nurse_10_day_3_Night + 1*ShiftsWorked_Nurse_10_day_4_Day + 1*ShiftsWorked_Nurse_10_day_4_Late_Night + 1*ShiftsWorked_Nurse_10_day_4_Night + 1*ShiftsWorked_Nurse_10_day_5_Day + 1*ShiftsWorked_Nurse_10_day_5_Late_Night + 1*ShiftsWorked_Nurse_10_day_5_Night + 1*ShiftsWorked_Nurse_10_day_6_Day + 1*ShiftsWorked_Nurse_10_day_6_Late_Night + 1*ShiftsWorked_Nurse_10_day_6_Night + 1*ShiftsWorked_Nurse_10_day_7_Day + 1*ShiftsWorked_Nurse_10_day_7_Late_Night + 1*ShiftsWorked_Nurse_10_day_7_Night + 1*ShiftsWorked_Nurse_10_day_

In [3]:
# Define hard constraints
for day in days:
    for shift in shifts:
        prob += lpSum([shifts_worked[nurse][day][shift] for nurse in nurses]) == shift_requirements[shift], f"ShiftRequirement_{day}_{shift}" # 各日の各シフトの要件を満たす制約を設定
for nurse in nurses:
    prob += lpSum([shifts_worked[nurse][day][shift] for day in days for shift in shifts]) == workday_requirements[nurse], f"WorkdayRequirement_{nurse}" # 各ナースの必要な勤務日数を満たす制約を設定

print(prob)# 変数，目的関数，最低条件を表示
    
# Define soft constraints
for nurse in nurses:
    for i in range(len(days)-1):
         if i < len(days) - 1:
             # 制約：ナースは深夜勤務を2日連続で行わない
            prob += shifts_worked[nurse][days[i]]['Late Night'] + shifts_worked[nurse][days[(i+1)%len(days)]]['Late Night'] <= 1, f"NoConsecutiveLateNights_{nurse}_{days[i]}"
            # 一晩勤務の後に翌日の日勤シフトに入らない制約を設定
            prob += shifts_worked[nurse][days[i]]['Late Night'] + shifts_worked[nurse][days[i+1]]['Day'] <= 1, f"NoLateNightToDayShift_{nurse}_{days[i]}" 
    for i in range(len(days)-2): # len(days)-1 を len(days)-2 に変更
        # 制約：ナースは3日連続で勤務しない（シフトは問わない）
        prob += lpSum([shifts_worked[nurse][days[i+j]][shift] for j in range(3) for shift in shifts]) <= 2, f"No3StraightDays_{nurse}_{days[i]}"  

print(prob)# 変数，目的関数，最低条件，オプション条件を表示

NurseSchedulingProblem:
MINIMIZE
1*ShiftsWorked_Nurse_10_day_10_Day + 1*ShiftsWorked_Nurse_10_day_10_Late_Night + 1*ShiftsWorked_Nurse_10_day_10_Night + 1*ShiftsWorked_Nurse_10_day_1_Day + 1*ShiftsWorked_Nurse_10_day_1_Late_Night + 1*ShiftsWorked_Nurse_10_day_1_Night + 1*ShiftsWorked_Nurse_10_day_2_Day + 1*ShiftsWorked_Nurse_10_day_2_Late_Night + 1*ShiftsWorked_Nurse_10_day_2_Night + 1*ShiftsWorked_Nurse_10_day_3_Day + 1*ShiftsWorked_Nurse_10_day_3_Late_Night + 1*ShiftsWorked_Nurse_10_day_3_Night + 1*ShiftsWorked_Nurse_10_day_4_Day + 1*ShiftsWorked_Nurse_10_day_4_Late_Night + 1*ShiftsWorked_Nurse_10_day_4_Night + 1*ShiftsWorked_Nurse_10_day_5_Day + 1*ShiftsWorked_Nurse_10_day_5_Late_Night + 1*ShiftsWorked_Nurse_10_day_5_Night + 1*ShiftsWorked_Nurse_10_day_6_Day + 1*ShiftsWorked_Nurse_10_day_6_Late_Night + 1*ShiftsWorked_Nurse_10_day_6_Night + 1*ShiftsWorked_Nurse_10_day_7_Day + 1*ShiftsWorked_Nurse_10_day_7_Late_Night + 1*ShiftsWorked_Nurse_10_day_7_Night + 1*ShiftsWorked_Nurse_10_day_

In [4]:
# Solve problem and print solution
prob.solve() # 問題を解く
print("Status:", LpStatus[prob.status]) # 問題のステータスを出力
for nurse in nurses:
    print("Nurse:", nurse)
    for day in days:
        for shift in shifts:
            if shifts_worked[nurse][day][shift].value() == 1:
                print(f"\t{day}: {shift}") # 割り当てられたシフトを出力

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/6bf4e474e6b84b03bdaa8a703e17c32e-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/6bf4e474e6b84b03bdaa8a703e17c32e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 305 COLUMNS
At line 2886 RHS
At line 3187 BOUNDS
At line 3488 ENDATA
Problem MODEL has 300 rows, 300 columns and 1680 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.03 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.06   (Wallclock seconds):       0.06

Status: Infeasible
Nurse: Nurse 1
	day 2: Night
	day 4: Day
	day 7: Day
	day 8: Day
Nurse: Nurse 2
	day 1: Night
	day 1: Late Night
	day 5: Day
	day 8: Day
	day 9: Day
Nurse: Nurse 3
	day 2: Late Night
	day 6: Day
	day 7: Night
	day 9: Day
	day